<a href="https://colab.research.google.com/github/ctate7163/radiometry/blob/main/radiometry.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Radiometry for exposure time calculation

This notebook estimates the camera design requirements

All quantities are in SI units unless stated otherwise.

Symbols used below.

L: radiance [W/sr/m^@2]
S: signal electrons [#e]
λ: wavelength [meters]


In [ ]:
import numpy as np
import matplotlib.pyplot as plt